# Navigation - Project 1

---
This notebook we train an agent based on the DQN - Algorithm describe on the following ["paper"](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf).

## 0| Environement
Load all required packages and the "Banana" [unity environement](https://github.com/Unity-Technologies/ml-agents).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import matplotlib.pyplot as plt
from _001_dqn_agent import Agent
from collections import deque

# Load Banana environement
env = UnityEnvironment(file_name="Banana.app")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## 1| DQN - Agent Training

In [5]:
def dqn(n_episodes=2000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, train=True):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    agent = Agent(state_size=37, action_size=4, seed=0)

    scores = []  # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start  # initialize epsilon

    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0

        while True:
            # Agent decision and interaction
            action = agent.act(state)
            env_info = env.step(action)[brain_name]

            # Feedback on action
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]

            # Update values
            if train:
                agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward

            if done:
                break

        scores_window.append(score)  # save most recent score
        scores.append(score)  # save most recent score
        eps = max(eps_end, eps_decay * eps)  # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(model.state_dict(), 'checkpoint_dqn.pth') # Save model

    return scores

Episode 100	Average Score: 1.28
Episode 200	Average Score: 3.31
Episode 300	Average Score: 8.81
Episode 400	Average Score: 12.41
Episode 500	Average Score: 16.05
Episode 600	Average Score: 16.52
Episode 700	Average Score: 16.47
Episode 800	Average Score: 16.07
Episode 900	Average Score: 16.88
Episode 1000	Average Score: 16.81
Episode 1100	Average Score: 16.22
Episode 1188	Average Score: 16.28

KeyboardInterrupt: 

In [ ]:
scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

## 2| DQN - Agent Run

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, seed=0, filename='checkpoint_dqn.pth')
scores = dqn(agent, n_episodes=100, train=False)

In [ ]:
env.close()